In [ ]:
# load dataset
from p_efficientnet_b7_v1 import define
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader

transforms = define.transform()

train_data = ImageFolder("./dataset/train", transform=transforms)
test_data = ImageFolder("./dataset/test", transform=transforms)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True)

In [ ]:
# define model
model = define.create_model()
model_desc = define.model_desc()
model_directory_name = define.__file__.replace("\\", "/").split("/")[-2]

In [ ]:
# create runner
import util

runner = util.Runner(model, default_dir=model_directory_name)
print(f"runner use {runner.device}")
runner.load(None) # load latest model

In [ ]:
import util

img, label = next(iter(train_loader))
labels = [ train_data.classes[x] for x in label]

def func(data, target, result):
    data = data.to('cpu')
    corr = [ test_data.classes[x] for x in target ]
    pred = [ test_data.classes[x] for x in result ]
    title = f"corr: {' * '.join(corr)}\npred: {' * '.join(pred)}"
    util.show2(data, title)
runner.test(test_loader, func)